In [ ]:
#%pip install selenium pandas mysql-connector-python streamlit matplotlib seaborn
#%pip install mysql-connector-python
#%pip install pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd




# Set up Selenium WebDriver
driver = webdriver.Chrome()
genres = ["biography", "western", "sci-fi", "musical", "war"]
IMDB_URL = "https://www.imdb.com/search/title/?title_type=feature&release_date=2024-01-01,2024-12-31"
driver.get(IMDB_URL)
time.sleep(5)


#define clicking next pages

def click_next_page():
    button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[2]/div/span/button')
    ActionChains(driver).move_to_element(button).perform()
    button.click()
    time.sleep(5)
    return click_next_page()

#convert k into 1000
def convert_to_number(value):
    if 'k' in value.lower():
        return int(float(value.lower().replace('k', '')) * 1000)
    return int(value)

#covert hours duration into minutes
def convert_duration_to_minutes(duration):
    try:
        parts = duration.split()
        hours = 0
        minutes = 0
        for part in parts:
            if 'h' in part:
                hours = int(part.replace('h', ''))
            elif 'm' in part:
                minutes = int(part.replace('m', ''))
        return hours * 60 + minutes
    except Exception as e:
        print(f"Error: {e}")
        return None

#scraping movie datas as per genres
for i in genres:
    url=f"{IMDB_URL}&genres={i}"
    # print(i)
    driver.get(url)
    time.sleep(5)
    
    #load pages
    while True:
        try:
            click_next_page()
        except:
            # print("No more pages")
            break
    
    #Empty list for data storage
    movie_titles = []
    movie_durations = []
    movie_ratings= []
    movie_vote_counts = []

    #Selection of movie container
    movie_container = driver.find_elements(By.XPATH,'//*[@id="__next"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/ul/li')                    
    
    #scraping movie data
    for movie in movie_container:
        try:
            #fetch movie title
            movie_title = movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[1]/a/h3').text
            #fetch movie rating
            movie_rating = movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[1]').text
            #fetch vote counts and save it in string format
            movie_vote_count = movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/span/div/span/span[2]').text.replace("(","").replace(")","")
            movie_vote_count= convert_to_number(movie_vote_count)
            #fetch movie duration and save it in minuts integer formt
            movie_duration = movie.find_element(By.XPATH,'./div/div/div/div[1]/div[2]/div[2]/span[2]').text
            movie_duration=convert_duration_to_minutes(movie_duration)
            
            #append the data in respective lists
            movie_titles.append(movie_title)
            movie_durations.append(movie_duration)
            movie_ratings.append(movie_rating)
            movie_vote_counts.append(movie_vote_count)
                    
        except:
            pass
    
    #create a DataFrame
    df = pd.DataFrame({"Title":movie_titles,
                       "Rating":movie_ratings,
                       "Votes":movie_vote_counts,
                       "Duration":movie_durations,
                       "Genre":f"{i}"})
    #save as a csv file
    df.to_csv(f"{i}.csv",index=False)
        
# Close the browser
driver.quit()



In [2]:
import os
import pandas as pd

# Specify the folder where your CSV files are located
folder_path = r"C:\Users\isbmy\.vscode\GUVI_Project_1"

# List to hold DataFrames
dataframes = []

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        # Read each CSV file into a DataFrame
        df = pd.read_csv(file_path)
        dataframes.append(df)

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
output_file = "combined_movies.csv"
combined_df.to_csv(output_file, index=False)

print(f"All CSV files have been combined and saved to {output_file}")


All CSV files have been combined and saved to combined_movies.csv


In [ ]:
import mysql.connector

# Connect to TiDB Cloud SQL
connection = mysql.connector.connect(
    user='3gjpMYnFyBFAYAb.root',
    password='J9foCvujVRIFORzR',
    host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',
    port='4000'
)

if connection.is_connected():
    print("Connected to TiDB Cloud SQL")
else:
    print("Connection failed")

In [ ]:
import csv


cursor = connection.cursor()

# Create database IMDB_2024 if it doesn't exist
cursor.execute("CREATE DATABASE IF NOT EXISTS IMDB_2024;")
cursor.execute("USE IMDB_2024;")

# Create table combined_movies
create_table_query = """
CREATE TABLE IF NOT EXISTS combined_movies (
    Title VARCHAR(255),
    Rating FLOAT,
    Votes INT,
    Duration INT,
    Genre VARCHAR(50)
);
"""
cursor.execute(create_table_query)

# Insert data from combined_movies.csv

with open('imdb_2024.csv', 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)  # Skip the header row
    insert_query = """
    INSERT INTO combined_movies (Title, Rating, Votes, Duration, Genre)
    VALUES (%s, %s, %s, %s, %s);
    """
    for row in csv_reader:
        cursor.execute(insert_query, row)

# Commit changes and close the connection
connection.commit()
cursor.close()
connection.close()

print("Data from imdb_2024.csv has been inserted into the IMDB_2024 database.")